In [1]:
%matplotlib inline

import cv2
import glob

import numpy as np
import matplotlib.pyplot as plt

In [2]:
class Perspective:
    def __init__(self, src, dst):
        self.warp_M = cv2.getPerspectiveTransform(src, dst)
        self.unwarp_M = cv2.getPerspectiveTransform(dst, src)
    
    def warp(self, img):
        size = (img.shape[1], img.shape[0])
        return cv2.warpPerspective(img, self.warp_M, img_size, flags=cv2.INTER_LINEAR)
    
    def unwarp(self, img):
        size = (img.shape[1], img.shape[0])
        return cv2.warpPerspective(img, self.unwarp_M, img_size, flags=cv2.INTER_LINEAR)

In [10]:
def camera_calibration(img_shape=(720, 1280)):
    img_list = glob.glob('camera_cal/*.jpg')
    
    for path in img_list:
        img = cv2.imread(path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)
        
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
            
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
    return mtx, dist

def combined_thresh(gray, s_channel, sobel_kernel=3, dir_kernel=15, sobel_thesh=(20, 100), 
                    mag_thresh=(30, 100), dir_thresh=(0.7, 1.3), s_thresh = (170, 255)):
    
    # abs sobel thresh
    gradx = abs_sobel_thresh(gray, orient='x')
    grady = abs_sobel_thresh(gray, orient='y')
    
    abs_binary = np.zeros_like(gradx)
    abs_binary[(gradx == 1) & (grady == 1)] = 1
    
    # mag thresh
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    sobelxy = np.sqrt(sobelx**2 + sobely**2)
    scaled = np.uint8(255*(sobelxy / np.max(sobelxy)))

    mag_binary = np.zeros_like(scaled)
    mag_binary[(scaled >= mag_thresh[0]) & (scaled <= mag_thresh[1])] = 1
    
    # dir thresh
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=dir_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=dir_kernel)
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    
    dir_binary =  np.zeros_like(absgraddir)
    dir_binary[(absgraddir >= dir_thresh[0]) & (absgraddir <= dir_thresh[1])] = 1
    
    # S channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel > s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    

    # combined binary
    combined_binary = np.zeros_like(abs_binary)
    combined_binary[(abs_binary == 1) | ((mag_binary == 1) & (dir_binary == 1)) | (s_binary == 1)] = 1
    
    return combined_binary

def abs_sobel_thresh(gray, orient='x', thresh=(20, 100)):
    
    if orient == 'x':
        abs_sobelx = np.abs(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    else:
        abs_sobelx = np.abs(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    scaled = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    binary = np.zeros_like(scaled)
    binary[(scaled > thresh[0]) & (scaled <= thresh[1])] = 1
    
    
    return binary

def window_mask(width, height, img_ref, center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

def find_line(warped):
    warped = warped * 255
    # window settings
    window_width = 50 
    window_height = 80 # Break image into 9 vertical layers since image height is 720
    margin = 100 # How much to slide left and right for searching
        
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # Sum quarter bottom of image to get slice, could use a different ratio
    l_sum = np.sum(warped[int(3*warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    r_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(warped.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(warped.shape[1]/2)
    
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(warped.shape[0]/window_height)):
        # convolve the window into the vertical slice of the image
        image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
        conv_signal = np.convolve(window, image_layer)
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin,warped.shape[1]))
        l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
        # Find the best right centroid by using past right center as a reference
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin,warped.shape[1]))
        r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
        # Add what we found for that layer
        window_centroids.append((l_center,r_center))

    # If we found any window centers
    if len(window_centroids) > 0:

        # Points used to draw all the left and right windows
        l_points = np.zeros_like(warped)
        r_points = np.zeros_like(warped)

        # Go through each level and draw the windows 	
        for level in range(0,len(window_centroids)):
            # Window_mask is a function to draw window areas
            l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
            r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
            # Add graphic points from window mask here to total pixels found 
            l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
            r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

        # Draw the results
        template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
        
        detected_line = np.zeros_like(warped)
        detected_line[(warped == 255) & (template == 255)] = 255
        
        middle = int(detected_line.shape[1]/2)
        left_detected = detected_line[:,:middle].nonzero()
        right_detected = detected_line[:,middle:].nonzero()

        left_fit = np.polyfit(left_detected[0], left_detected[1], 2)
        right_fit = np.polyfit(right_detected[0], (middle + right_detected[1]), 2)

    
    # If no window centers found, just display orginal road image
    else:
        output = np.array(cv2.merge((warped,warped,warped)),np.uint8)
    
    return left_fit, right_fit

def calculate_curvature(left_fit, right_fit, img_shape=(720, 1280)):
    ym_px = 30 / 720 # Real world metres per y pixel
    xm_px = 3.7 / 700 # Real world metres per x pixel   

    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]
    
    left_fit_cr = np.polyfit(ploty * ym_px, left_fitx * xm_px, 2)
    right_fit_cr = np.polyfit(ploty * ym_px, right_fitx * xm_px, 2)

    y_max = np.max(ploty)
    left_curverad = ((1 + (2 * left_fit_cr[0] * y_max * ym_px + left_fit_cr[1]) ** 2) ** 1.5) / np.absolute(2 * left_fit_cr[0])
    right_curverad = ((1 + (2 * right_fit_cr[0] * y_max * ym_px + right_fit_cr[1]) ** 2) ** 1.5) / np.absolute(2 * right_fit_cr[0])

    return left_curverad, right_curverad


def plot_lane_area(img, left_fit, right_fit):
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]
    
    # get poly coordinate
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    
    area = np.zeros_like(img).astype(np.uint8)
    cv2.fillPoly(area, np.int_([pts]), (0,255, 0))
    
    # unwarp 
    area = perspective.unwarp(area)
    
    # add area to image
    result = cv2.addWeighted(img, 1, area, 0.3, 0)
    return result

def calculate_offset(left_fit, right_fit, img_shape=(720, 1280)):
    lane_width = 3.7 / 700
    height, width = img_shape
    
    left_point = left_fit[0] * height ** 2 + left_fit[1] * height + left_fit[2]
    right_point = right_fit[0] * height ** 2 + right_fit[1] * height + right_fit[2]
    mid_point = np.mean([left_point, right_point])
    
    offset = ((width / 2) - mid_point) * lane_width
    
    return offset

In [9]:
def process(img, img_size=(720, 1280)):
    alpha = 0.1
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.undistort(img, mtx, dist)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    # combined thresh
    thresh = combined_thresh(gray, hls[:, :, 2], sobel_kernel=5, dir_kernel=15, sobel_thesh=(50, 255), 
                             mag_thresh=(50, 255), dir_thresh=(0.7, 1.3), s_thresh = (170, 255))

    # warp thresh image
    warped_line = perspective.warp(thresh)
    
    # ignore side value
    ignore = 100
    warped_line[:ignore] = 0
    warped_line[warped_line.shape[1] - ignore:] = 0
    
    
    left_fit, right_fit = find_line(warped_line)

    left_rad, right_rad = calculate_curvature(left_fit, right_fit, img_size)
    rad = np.mean([left_rad, right_rad])
    offset = calculate_offset(left_fit, right_fit)
    
    result = plot_lane_area(img, left_fit, right_fit)
    
    # put text
    cv2.putText(result, 'Curvature Radius: {}m'.format(int(rad)), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    cv2.putText(result, 'Lane Offset: {}m'.format(int(offset)), (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    
    result = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)
    
    return result

In [6]:
nx = 9
ny = 6
img_shape = (720, 1280)

objp = np.zeros((nx*ny,3), np.float32)
objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

objpoints = [] 
imgpoints = []

mtx, dist = camera_calibration(img_shape)

img_size = (img_shape[1], img_shape[0])
offset = 300
src = np.float32([[215, 700], [590, 450], [690, 450], [1100, 700]])
dst = np.float32([[offset, img_size[1]], [offset, 0],
                  [img_size[0]-offset, 0], [img_size[0]-offset, img_size[1]]])
perspective = Perspective(src, dst)

In [11]:
for path in glob.glob("test_images/*.jpg"):
    img = cv2.imread(path)
    result = process(img) #input BGR image
    save_path = path.replace("test_images", "output_images")
    cv2.imwrite(save_path, result)

In [13]:
from moviepy.editor import VideoFileClip

white_output = 'project_video_output.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process) #NOTE: this function expects color images!!

%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [06:19<00:00,  3.37it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

Wall time: 6min 19s
